# Primera entrega ML

### O.S.C. - Optimización del sistema de cultivo 

### Tema

La tésis consiste en cruzar 8 parentales salvajes de 2 variedades diferentes de pepinos (4 de un tipo y 4 de otro), y generar una variedad que adquiera, tolere y saque más partido de los potásios de la tierra que las variedades comerciales de hoy en dia, para reducir notablemente la contaminación y los costes de producción. 

Siguiendo esto, se plantea el siguiente experimento: Tenemos 6 plantas de cada variedad (128 variedades), y el primer mes se cultivan las 6 con las mismas condiciones. Pasado el primer mes se reduce la cantidad de potásio de 3 de esas 6 plantas al 50%. 
Pasados 20 dias desde la modificación en el potasio, se realiza el primer análisis de las plantas (pesos frescos, pesos secos, y demás componentes). Y pasados 160/170 días se extraen las plantas para su completo análisis. Se utiliza una variedad comercial como referencia de control.

#### El objetivo del modelo predictivo es; 
Mediante los valores obtenidos en el primer análisis (20 dias después del cambio en la tasa de nitrogeno en 3 de las 6 plantas de cada variedad) PREDECIR si esas plantas sirven para continuar con el experimento, sin necesidad de esperar esos 5 o 6 meses para que las plantas acaben de crecer. 

De esta manera se optimizarán notablemente timepos y costes del proyecto. 

### Datasets y fuentes alternativas de datos


Utilizaremos un dataset con los registros de cada una de las plantas en según qué momento del experimento.

Se plantearán 3 targets a predecir con modelos de regresión. Explorando varias opciones para valorar cual de los modelos es más eficaz para con nuestro dataset. Además generaremos una nueva feature a partir de valores clave para utilizarla como target en un modelo de clasificación, y estimar si la variedad a predecir tendrá éxito en el experimento, utilizando las métricas correspondientes a cada modelo.

In [1]:
import pandas as pd
import re
import openpyxl

In [3]:
workbook = openpyxl.load_workbook("Datos_estadística.xlsx", data_only=True)
sheet = workbook.active 

# Extraer encabezados de la primera fila
headers = [cell.value for cell in sheet[1]]

# Extraer datos de las filas siguientes
data = []
for row_idx in range(2, sheet.max_row + 1): # Desde la segunda fila hasta el final
    row_values = [sheet.cell(row=row_idx, column=col_idx).value for col_idx in range(1, sheet.max_column + 1)]
    data.append(row_values)

# Crear el DataFrame de Pandas
df = pd.DataFrame(data, columns=headers)

# Eliminamos la columnas llena de "None"
df = df.iloc[:,:53]

# Limpiamos de espacios problemáticos, y los cambiamos por "_" para trabajar mejor con los nombres de las columnas.
column_names = df.columns
new_column_names = []

for col in column_names:
    # Eliminar espacios al principio y al final (strip())
    # Reemplazar cualquier secuencia de uno o más espacios (s+) por un guion bajo (_)
    # Usamos re.sub para manejar múltiples espacios como uno solo
    cleaned_col = re.sub(r'\s+', '_', col.strip())
    new_column_names.append(cleaned_col)

df.columns = new_column_names

df.head(10)

,Block,Position,Code,Treatment,Replicate,Peso_de_muestra_vegetal_(g),Cantidad_de_N_en_muestra_vegetal_(g),Cantidad_de_C_en_muestra_vegetal_(g),Porcentaje_N_de_la_muestra_vegetal,Porcentaje_C_de_la_muestra_vegetal,...,Biomasa_total_fresca,Biomasa_total_seca,Harvest_Index_S_(His),Root_Index_(Ri),CHLOR,FLV,ANT,NBI,NDVI,PhipII
0,2,375,1,100,1,0.0843,0.00214,0.0306,2.53338,36.27933,...,905.650,166.889,0.020906,0.085086,30.626,0.568,0.214,53.96,0.626689,0.856
1,2,181,1,100,2,0.0860,0.00239,0.0325,2.78006,37.77989,...,1193.085,201.468,0.033593,0.085373,30.129,0.492,0.198,61.23,0.616728,0.790
2,1,246,1,100,3,0.0846,0.00205,0.0327,2.42762,38.70086,...,856.936,146.899,0.032669,0.090538,25.620,0.534,0.208,47.97,0.644530,0.861
3,2,357,2,100,1,0.0851,0.00180,0.0318,2.11269,37.40876,...,838.955,136.202,0.097664,0.154183,29.626,0.549,0.215,54.01,0.620586,0.859
4,2,241,2,100,2,0.0857,0.00173,0.0336,2.01700,39.21480,...,776.498,147.557,0.082389,0.112499,33.623,0.523,0.196,64.33,0.644228,0.819
5,1,204,2,100,3,0.0857,0.00215,0.0321,2.51177,37.49667,...,884.680,167.873,0.064769,0.095906,34.343,0.555,0.174,61.85,0.650458,0.843
6,2,371,3,100,1,0.0833,0.00196,0.0309,2.35061,37.05629,...,723.978,139.495,0.052296,0.078856,22.457,0.628,0.240,35.78,0.587634,0.799
7,2,341,3,100,2,0.0857,0.00176,0.0328,2.05373,38.23354,...,861.151,171.476,0.080338,0.129464,27.422,0.576,0.219,47.61,0.632312,0.786
8,2,329,3,100,3,0.0829,0.00185,0.0314,2.22803,37.90747,...,853.020,156.391,0.110563,0.095913,25.456,0.534,0.210,47.66,0.600110,0.773
9,1,50,4,100,1,0.0840,0.00187,0.0260,2.23149,30.95969,...,864.176,133.115,0.019645,0.142734,25.261,0.667,0.198,37.88,0.677189,0.857
